In [1]:
import pandas as pd
import re
import string
'''
전처리(Preprocessing)는 필요 없는 요소를 제외하여 주요 키워드들만 사용하기 위한 것으로 
자간 · 행간 등 처리, 문장 단위 구분, 불용어 처리, 품사 태깅(Tagging)을 통한 불용 품사 처리, 문서 복구 등을 수행
'''

news = pd.read_csv('../data/daum_news.csv', encoding='utf-8-sig')

In [2]:
def news_body_cleanser(news_body : str) -> str :
    """
    뉴스기사 내용을 가져와 문장 부호와 불필요한 내용을 제거하는 함수입니다.\n
    전처리된 뉴스기사 내용인 문자열(String) 객체를 반환합니다.
    """

    # 괄호 안의 모든 문자 지정
    parenthesize_pattern = re.compile(r"\([^)]+\)|\[[^]]+\]|\<[^>]+\>|\{[^}]+\}|\（[^）]+\）|\［[^［]+\］|\〈[^〈]+\〉|\《[^《]+\》|\＜[^＜]+\＞|\｛[^｛]+\｝")
    # 평균자책점 같이 소수점이 들어가는 숫자 지정
    decimal_point_pattern = re.compile(r"[\d]+\.[\d]+")

    # 기자의 이메일, 이름을 지정
    email_pattern = re.compile(r"[a-zA-Z0-9+-_./]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
    reporter_pattern = re.compile(r"([가-힣]{2,5} 기자)|([가-힣]{2,5}기자)")
    # 숫자 지정
    not_hangul_num_pattern = re.compile(r"[0-9]")

    news_body = parenthesize_pattern.sub("", news_body).strip()
    news_body = decimal_point_pattern.sub("", news_body).strip()
    news_body = email_pattern.sub("", news_body).strip()
    news_body = reporter_pattern.sub("", news_body).strip()
    news_body = not_hangul_num_pattern.sub("", news_body).strip()

    # translate() 메서드를 사용해 각종 기호, 줄바꿈 및 문장부호를 제거
    symbols = string.punctuation.replace(".", "").replace("?", "").replace("!", "") + "·ㆍ■◆△▷▶▼�“”‘’…※↑↓▲☞ⓒ⅔"
    news_body = news_body.translate(str.maketrans("", "", symbols))
    news_body = news_body.replace("\n", "")
    news_body = news_body.replace("마이데일리.", "")
    news_body = news_body.replace("무단전재재배포 금지", "")

    # 결과 값 반환
    return news_body

In [3]:
text = news_body_cleanser(news['Content'][6])

In [21]:
from konlpy.tag import Mecab

def delete_stop_words(news_body : str):
    m = Mecab()
    
    file = open('../data/korean_stop_words.txt', 'r', encoding='utf-8')
    kstop_words = file.readlines()[0]
    kstop_words = set(kstop_words.split(' '))

    tokens = m.morphs(news_body)

    return [word for word in tokens if not word in kstop_words]

In [24]:
for n in news['Content'][:5]:
    cleaned_news = news_body_cleanser(n)
    print(delete_stop_words(cleaned_news))

['위기', '라는', '팀', '이나', '찾아온다', '.', '감', '부딪히', '중요', '현대', '건설', '수원', '체육관', '열린', '도드', '람', 'V', '리그', '라운드', 'KGC', '삼공사', '경기', '세트', '스코어', '역전승', '거두', '고', '개막', '연승', '질주', '.', '양효진', '득점', '활약', '승리', '이끌', '었', '.', '경기', '인터뷰', '오늘', '힘든', '경기', '.', '마지막', '이겨서', '다행', '라고', '한다', '라고', '승리', '소감', '밝혔', '.', '현대', '건설', '이날', '승리', '기', '쉽', '경기', '.', '세트', '인삼', '공사', '세트', '스코어', '끌려가', '경기', '.', '세트', '후반', '인삼', '공사', '외국인', '선수', '엘리자벳', '부상', '당하', '면서', '현대', '건설', '힘겹', '역전승', '거둘', '었', '.', '상대', '준비', '라고', '양효진', '도', '리듬', '맞', '았', '.', '전체', '움직임', '이나', '매끄럽', '던', '평소', '보다', '느낌', '.', '선수', '역할', '고', '이겨', '서', '마지막', '이길', '었', '다고', '한다', '.', '면', '고비', '넘겼', '다고', '볼', '라고', '았', '던', '경기력', '아쉬워했', '.', '양효진', '연승', '라는', '부담', '면', '된다고', '한다', '라고', '강조', '대한', '두려움', '생기', '면', '될', '.', '작년', '도', '그랬', '.', '계속', '이기', '면', '니', '계속', '이기', '고', '.', '풀', '할지', '고민', '해야', '정말', '강팀', '될', '.', '오늘', '상황', '왔', '이겨', '나갈', '으면', '겠', '라고